In [26]:
!pip install langchain
!pip install langchain-openai
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken
!pip install typing-inspect==0.8.0
!pip install typing_extensions==4.5.0
!pip install pydantic==1.10.11
!pip install docx2txt
!pip install --upgrade --quiet  langchain-core langchain-community langchain-openai

  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.111.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
openai 1.26.0 requires typing-extensions<5,>=4.7, but you have typing-extensions 4.5.0 which is incompatible.
sqlalchemy 2.0.25 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
unstructured-client 0.22.0 requires typing-extensions>=4.7.1, but you have typing-extensions 4.5.0 which is incompatible.
unstructured-client 0.22.0 requires typing-inspect>=0.9.0, but you have typing-inspect 0.8.0 which is incompatible.


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3973 sha256=033050a26c1662937ffff4a55337212398c0dc2ba2fa7e623100e32faa390f0a
  Stored in directory: c:\users\hooda\appdata\local\pip\cache\wheels\0f\0e\7a\3094a4ceefe657bff7e12dd9592a9d5b6487ef4338ace0afa6
Successfully built docx2txt


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-client 0.22.0 requires typing-inspect>=0.9.0, but you have typing-inspect 0.8.0 which is incompatible.


In [29]:
!pip install --upgrade typing-extensions
!pip install typing-inspect

In [32]:
from langchain_openai import OpenAIEmbeddings

In [33]:
from PyPDF2 import PdfReader
from typing import Iterator
from langchain_openai import OpenAIEmbeddings
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
import docx2txt

In [34]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [35]:
pdfreader = PdfReader('Anubhavs M Resume.pdf')

In [36]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [37]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)
cleaned_texts = [chunk.replace("\n", " ") for chunk in texts]
cleaned_texts = [chunk.replace("_", "") for chunk in cleaned_texts ]
# cleaned_texts = re.sub('_{2,}', ' ', cleaned_texts)
print(cleaned_texts)

['ANUBHAV HOODA LinkedIn | +91- 9911865100 | g.dev | hoodaanubhav@gmail.com | GitHub Skills  • C++ | Python | Java | JavaScript | TypeScript | C | MongoDB | MSSQL | Node | Express | React | Vue | NoSQL | Git • Deep Learning | Natural Language Processing(NLP) | Scikit-learn | TensorFlow | PyTorch | Feature Engineering • Data Transformation | AWS EC2| Cloud Computing | Unit Testing | Lambda | OOP | Unity 2D | Performance Evaluation Experience  Artiﬁcial Intelligence and  Machine Learning Head Google Developer Students Club 07/2023 - Current • As a mentor, I support and guide my college juniors in machine learning,', 'Artiﬁcial Intelligence and  Machine Learning Head Google Developer Students Club 07/2023 - Current • As a mentor, I support and guide my college juniors in machine learning, assisting them with their initial machine learning projects. • Facilitated exposure by organizing multiple TED Talks and inviting industry experts as guest speakers on behalf of the Google Developer Stud

In [38]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [39]:
document_search = FAISS.from_texts(cleaned_texts, embeddings)

In [40]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [41]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [ ]:
query = "What are the skills?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

In [ ]:
query = "What are the projects?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

In [42]:
query = "You are a seasoned recruiter tasked with evaluating the suitability of a candidate for the position of Back End Software Engineer based on their resume. Your assessment should be thorough and insightful, reflecting both the candidate's qualifications and the specific requirements outlined in the job description. Consider the candidate's technical skills, experience, and potential cultural fit within the organization. Provide a detailed analysis, including a numerical rating from 1 to 10 along with a comprehensive explanation justifying the score. Take into account both the strengths and areas for improvement highlighted in the resume and how well they align with the responsibilities and expectations outlined in the job description. Aim for a holistic assessment that goes beyond surface-level evaluation."
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\nBased on the candidate\'s resume, it is evident that they have a strong background in machine learning, with a Bachelor\'s degree in Computer Engineering and Artificial Intelligence and experience as an Artificial Intelligence and Machine Learning Head at Google Developer Students Club. Their technical skills include proficiency in languages such as C++, Python, and Java, as well as experience with deep learning, natural language processing, and cloud computing.\n\nIn terms of experience, the candidate has worked on various projects, including the development of a CNN model for identification and extraction of Forward Error Correction (FEC) schemes and a large language model to tackle complex prompts. They have also facilitated workshops and mentored students through machine learning projects, emphasizing communication, collaboration, and problem-solving skills.\n\nFrom a cultural fit perspective, the candidate\'s experience as a team leader at Viral Fission and their statement of "

In [ ]:
text_extractor = docx2txt.TextExtractor()
with open("job_fit.docx", "rb") as f:
    job_description = text_extractor.extract(f)

In [18]:
# job_description = """
#     The Back End Software Engineer is responsible for the following:
#     • Develop server-side logic, definition and maintenance of the central database, and ensure high performance and responsiveness to requests from the front-end developers.
#     • Integrate user-facing elements developed by front-end developers with server-side applications.
#     • Collaborate with front-end developers, customers, users and Product Managers to establish objectives and design functional, cohesive codes to enhance the user experience.
#     • Keep abreast of novel technical concepts and markets.
#     • Provide technical leadership and documentation to developers and stakeholders.
#     • Apply usability procedures and principles as defined at the project or Product Line level or through customer input.
#     • Build prototypes, products and systems that meet the project quality standards and requirements.
#     • Contribute to and support re-use through common components that are well documented and tested
#     """

In [24]:
query = f"Resume:\n{cleaned_texts}\n\nJob Description:\n{job_description}\n\n Analyze how well the resume matches the job description, \
        then Analyze the resume and provide an estimate percentage of how well it matches the job description\
        Additionally, provide critiques on the resume:"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\n\nBased on the job description and the resume provided, the resume appears to match the job description quite well. The resume highlights experience and skills in relevant areas such as machine learning, artificial intelligence, and programming languages like Python, Java, and JavaScript. Additionally, the resume showcases experience in mentoring and collaborating with others, which are important skills for a back end software engineer. Overall, I would estimate that the resume matches the job description at around 85-90%.\n\nOne critique of the resume is that it could benefit from including more specific examples of projects or accomplishments, particularly in the experience section. This would help provide more concrete evidence of the candidate\'s skills and experiences. Additionally, the resume could benefit from being more concise and organized. Some sections, such as "Key Achievements" and "Projects," could be combined or further broken down into sub-sections to make the resum

In [23]:
def analyze_resume_job_match(cleaned_texts, job_description):    
    query =  f"Resume:\n{cleaned_texts}\n\nJob Description:\n{job_description}\n\n Analyze how well the resume matches the job description, \
        then Analyze the resume and provide an estimate percentage of how well it matches the job description\
        Additionally, provide critiques on the resume:"
    docs = document_search.similarity_search(query)
    chain.run(input_documents=docs, question=query)

In [ ]:
analyze_resume_job_match(cleaned_texts,job_description)